## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
keras = tf.keras

## Meus dados WIN

In [2]:
path = r'C:\Users\adria\1Python\Meu_novo_projeto\WIN_N_1K\WIN_N_1k.csv'
raw = pd.read_csv(path, index_col=0, parse_dates=True)

In [3]:
df = pd.DataFrame()
df['close'] = raw['close']

In [4]:
df['r'] = df['close'].pct_change()

In [5]:
df = df.dropna()

In [6]:
series = df['r'].values

In [7]:
time = np.arange(len(series))

In [8]:
split_time = 80000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

## Moving Average

In [9]:
def moving_average_forecast(series, window_size):
  """Forecasts the mean of the last few values.
     If window_size=1, then this is equivalent to naive forecast
     This implementation is *much* faster than the previous one"""
  mov = np.cumsum(series)
  mov[window_size:] = mov[window_size:] - mov[:-window_size]
  return mov[window_size - 1:-1] / window_size

In [10]:
moving_avg = moving_average_forecast(series, 5)[split_time - 5:]

In [11]:
diff_series = (series[10:] - series[:-10])
diff_time = time[10:]

In [12]:
diff_moving_avg = moving_average_forecast(diff_series, 5)[split_time - 10 - 5:]

In [13]:
diff_moving_avg_plus_past = series[split_time - 10:-10] + diff_moving_avg

In [14]:
keras.metrics.mean_absolute_error(x_valid, diff_moving_avg_plus_past).numpy()

0.0006491040976419816

In [15]:
keras.metrics.mean_squared_error(x_valid, diff_moving_avg_plus_past).numpy()

1.0431952563439955e-06

## DesRetorne os dados e verifique

In [16]:
df2 = pd.DataFrame()

In [17]:
df2['diff_moving_avg_plus_past'] = diff_moving_avg_plus_past

In [18]:
df2['x_valid'] = x_valid

In [19]:
df2

,diff_moving_avg_plus_past,x_valid
0,-0.000043,0.000264
1,0.000097,-0.000352
2,-0.001012,0.000220
3,0.000229,-0.000308
4,-0.000071,-0.000748
...,...,...
19993,-0.000009,-0.000044
19994,0.000122,0.000044
19995,-0.000044,0.000000
19996,-0.000078,-0.000044


In [20]:
df.iloc[79998:80003]

,close,r
time,,
2022-08-19 11:58:00,113585.0,-0.000484
2022-08-19 11:59:00,113630.0,0.000396
2022-08-19 12:00:00,113660.0,0.000264
2022-08-19 12:01:00,113620.0,-0.000352
2022-08-19 12:02:00,113645.0,0.000220


### desfazendo o x_valid

In [21]:
ultimo_preco_fechamento = 113630.0

In [22]:
df2['preco_valid'] = (1 + df2['x_valid']).cumprod()

In [23]:
df2['preco_valid'] = ultimo_preco_fechamento * df2['preco_valid']

In [24]:
df2

,diff_moving_avg_plus_past,x_valid,preco_valid
0,-0.000043,0.000264,113660.0
1,0.000097,-0.000352,113620.0
2,-0.001012,0.000220,113645.0
3,0.000229,-0.000308,113610.0
4,-0.000071,-0.000748,113525.0
...,...,...,...
19993,-0.000009,-0.000044,114915.0
19994,0.000122,0.000044,114920.0
19995,-0.000044,0.000000,114920.0
19996,-0.000078,-0.000044,114915.0


### desr a previsão

In [25]:
df2['desr_prev'] = (1 + df2['diff_moving_avg_plus_past']).cumprod()

In [26]:
df2['desr_prev'] = ultimo_preco_fechamento * df2['desr_prev']

In [27]:
df2

,diff_moving_avg_plus_past,x_valid,preco_valid,desr_prev
0,-0.000043,0.000264,113660.0,113625.057722
1,0.000097,-0.000352,113620.0,113636.062000
2,-0.001012,0.000220,113645.0,113521.048253
3,0.000229,-0.000308,113610.0,113547.024374
4,-0.000071,-0.000748,113525.0,113538.977604
...,...,...,...,...
19993,-0.000009,-0.000044,114915.0,114766.971081
19994,0.000122,0.000044,114920.0,114780.954263
19995,-0.000044,0.000000,114920.0,114775.959618
19996,-0.000078,-0.000044,114915.0,114766.970420


In [28]:
df2['diff'] = df2['preco_valid']-df2['desr_prev']

In [29]:
df2

,diff_moving_avg_plus_past,x_valid,preco_valid,desr_prev,diff
0,-0.000043,0.000264,113660.0,113625.057722,34.942278
1,0.000097,-0.000352,113620.0,113636.062000,-16.062000
2,-0.001012,0.000220,113645.0,113521.048253,123.951747
3,0.000229,-0.000308,113610.0,113547.024374,62.975626
4,-0.000071,-0.000748,113525.0,113538.977604,-13.977604
...,...,...,...,...,...
19993,-0.000009,-0.000044,114915.0,114766.971081,148.028919
19994,0.000122,0.000044,114920.0,114780.954263,139.045737
19995,-0.000044,0.000000,114920.0,114775.959618,144.040382
19996,-0.000078,-0.000044,114915.0,114766.970420,148.029580


In [31]:
df2['diff'].mean()

40.13865885362546